<a href="https://colab.research.google.com/github/matheusmf1/ECM514_Data_Science/blob/master/NLP_IMDB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Análise de Sentimentos de avaliações de filmes a portir de reviews do IMDB

- negativo
- positivo

In [1]:
import nltk
nltk.download('popular')
import pandas as pd
import numpy as np

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/cmudict.zip.
[nltk_data]    | Downloading package gazetteers to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gazetteers.zip.
[nltk_data]    | Downloading package genesis to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/genesis.zip.
[nltk_data]    | Downloading package gutenberg to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/gutenberg.zip.
[nltk_data]    | Downloading package inaugural to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/inaugural.zip.
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/movie_reviews.zip.
[nltk_data]    | Downloading package names to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/names.zip.
[nltk_data]    | Downloading package shakespeare to /root/nltk_data...
[nlt

# DataSet IMDB

In [2]:
df_imdb = pd.read_csv( '/content/drive/My Drive/Colab Notebooks/Data Science Class/DataSets/imdb-reviews-pt-br.csv' )
df_imdb.head()

,id,text_en,text_pt,sentiment
0,1,Once again Mr. Costner has dragged out a movie...,"Mais uma vez, o Sr. Costner arrumou um filme p...",neg
1,2,This is an example of why the majority of acti...,Este é um exemplo do motivo pelo qual a maiori...,neg
2,3,"First of all I hate those moronic rappers, who...","Primeiro de tudo eu odeio esses raps imbecis, ...",neg
3,4,Not even the Beatles could write songs everyon...,Nem mesmo os Beatles puderam escrever músicas ...,neg
4,5,Brass pictures movies is not a fitting word fo...,Filmes de fotos de latão não é uma palavra apr...,neg


In [3]:
df_imdb_pt = df_imdb[ ['text_pt', 'sentiment'] ]
df_imdb_pt.head()

,text_pt,sentiment
0,"Mais uma vez, o Sr. Costner arrumou um filme p...",neg
1,Este é um exemplo do motivo pelo qual a maiori...,neg
2,"Primeiro de tudo eu odeio esses raps imbecis, ...",neg
3,Nem mesmo os Beatles puderam escrever músicas ...,neg
4,Filmes de fotos de latão não é uma palavra apr...,neg


## Vamos pegar o neg e pos e substituir por 0 e 1


In [4]:
df_imdb_pt.sentiment.unique()

array(['neg', 'pos'], dtype=object)

In [5]:
df_imdb_pt.sentiment.replace( { 'neg': 0, 'pos': 1 }, inplace=True )
df_imdb_pt.head()

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:6746: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._update_inplace(new_data)


,text_pt,sentiment
0,"Mais uma vez, o Sr. Costner arrumou um filme p...",0
1,Este é um exemplo do motivo pelo qual a maiori...,0
2,"Primeiro de tudo eu odeio esses raps imbecis, ...",0
3,Nem mesmo os Beatles puderam escrever músicas ...,0
4,Filmes de fotos de latão não é uma palavra apr...,0


In [6]:
df_imdb_pt.text_pt[0]

'Mais uma vez, o Sr. Costner arrumou um filme por muito mais tempo do que o necessário. Além das terríveis seqüências de resgate no mar, das quais há muito poucas, eu simplesmente não me importei com nenhum dos personagens. A maioria de nós tem fantasmas no armário, e o personagem Costers é realizado logo no início, e depois esquecido até muito mais tarde, quando eu não me importava. O personagem com o qual deveríamos nos importar é muito arrogante e superconfiante, Ashton Kutcher. O problema é que ele sai como um garoto que pensa que é melhor do que qualquer outra pessoa ao seu redor e não mostra sinais de um armário desordenado. Seu único obstáculo parece estar vencendo Costner. Finalmente, quando estamos bem além do meio do caminho, Costner nos conta sobre os fantasmas dos Kutchers. Somos informados de por que Kutcher é levado a ser o melhor sem pressentimentos ou presságios anteriores. Nenhuma mágica aqui, era tudo que eu podia fazer para não desligar uma hora.'

# Vamos deixar tudo em minúsculo

In [8]:
df_imdb_pt['text_pt'] = df_imdb_pt['text_pt'].apply( str.lower )
df_imdb_pt.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,text_pt,sentiment
0,"mais uma vez, o sr. costner arrumou um filme p...",0
1,este é um exemplo do motivo pelo qual a maiori...,0
2,"primeiro de tudo eu odeio esses raps imbecis, ...",0
3,nem mesmo os beatles puderam escrever músicas ...,0
4,filmes de fotos de latão não é uma palavra apr...,0


# Modelo de referência

In [10]:
x = df_imdb_pt['text_pt'].values
x

array(['mais uma vez, o sr. costner arrumou um filme por muito mais tempo do que o necessário. além das terríveis seqüências de resgate no mar, das quais há muito poucas, eu simplesmente não me importei com nenhum dos personagens. a maioria de nós tem fantasmas no armário, e o personagem costers é realizado logo no início, e depois esquecido até muito mais tarde, quando eu não me importava. o personagem com o qual deveríamos nos importar é muito arrogante e superconfiante, ashton kutcher. o problema é que ele sai como um garoto que pensa que é melhor do que qualquer outra pessoa ao seu redor e não mostra sinais de um armário desordenado. seu único obstáculo parece estar vencendo costner. finalmente, quando estamos bem além do meio do caminho, costner nos conta sobre os fantasmas dos kutchers. somos informados de por que kutcher é levado a ser o melhor sem pressentimentos ou presságios anteriores. nenhuma mágica aqui, era tudo que eu podia fazer para não desligar uma hora.',
       'est

In [11]:
y = df_imdb_pt['sentiment'].values
y

array([0, 0, 0, ..., 1, 1, 1])